## קוד מבוא

In [11]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import adjustText as aT

In [12]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## פונקציות

In [13]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [14]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [15]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [16]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [17]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [18]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


## 

### העלת מידע

In [19]:
stat=up_load_shp(r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\חומר גלם\stat_11_cbs_2020_with_jtmt_fix.shp')

In [20]:
stat_in_taz=stat.loc[stat['type_stat']=='in_taz']

In [21]:
stat=stat.loc[stat['type_stat']!='in_taz']

In [22]:
taz=up_load_shp(r'W:\Model Versions\4.0\TAZ\TAZ_V4_230518_Published.shp')

In [23]:
path=r'W:\Data\GIS\BaseLayers\בנטל - מפי\BNTL 2022\Jerusalem Transportation.gdb'
bld=up_load_gdb(path,'BLDG_Clip')

In [24]:
path=r'W:\Data\GIS\BaseLayers\בנטל - מפי\BNTL 2022\Jerusalem Transportation.gdb'
bld_poi=up_load_gdb(path,'POI_BLDG_Clip')

In [25]:
taz_not_relevant=up_load_df(r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\חומר גלם','taz_not_relevant_for_pop_2020')

In [26]:
taz_not_relevant=list(taz_not_relevant.Taz_num)

### שכבת בניינים

In [27]:
bld=bld.merge(bld_poi[[ 'FCODE', 'USG_GROUP', 'USG_CODE','USG_SP_NM_LTN','BLDG_ID']],how='left',left_on='UNIQ_ID',right_on='BLDG_ID')

In [28]:
bld=bld.loc[(bld['FCODE_y'].isna())|(bld['USG_CODE']==7600)] #זה קוד לבניינים מעורב שימושים אני מניח שעדיף לשים בניינים מיותרים מאשר הפוך

In [29]:
bld['bld_area']=bld.area

In [30]:
bld_point=make_point(bld).fillna(0)

In [31]:
bld_point.loc[bld_point['HEIGHT']<=0,'HEIGHT']=bld_point['HI_PNT_Z']-bld_point['HT_LAND']

In [32]:
ceiling_height=3

In [33]:
bld_point['bld_m3']=(bld_point['HEIGHT']/ceiling_height).astype(int)*bld_point['bld_area'].astype(int)

In [34]:
bld_point.loc[(bld_point['HEIGHT']-ceiling_height)<=0,'bld_m3']=bld_point['bld_area']

### הורדת מבנים לא רלוונטים

In [35]:
bld_point=gpd.sjoin(bld_point,taz.loc[(~taz['Taz_num'].isin(taz_not_relevant))&(taz['Palestinia']==0)])

In [36]:
col=['bld_m3','centroid','Taz_num']

In [37]:
bld_point=bld_point[col]

In [38]:
bld_point=gpd.sjoin(bld_point,stat[['STAT','geometry']])

In [39]:
bld_point['bld_m3']=bld_point['bld_m3'].astype(int)

### יצירת מאה אחוז לכל א"ס

In [40]:
sum_bld_m3=pd.pivot_table(bld_point,index=['Taz_num','STAT'],aggfunc=sum)[['bld_m3']].reset_index()

In [41]:
sum_bld_m3_by_stat=sum_bld_m3.pivot_table(index='STAT',aggfunc=sum)[['bld_m3']].rename(columns={'bld_m3':'bld_m3_stat'}).reset_index()

In [42]:
sum_bld_m3=sum_bld_m3.merge(sum_bld_m3_by_stat,on='STAT')

In [43]:
sum_bld_m3['precent_of_stat_data']=sum_bld_m3['bld_m3']/sum_bld_m3['bld_m3_stat']

In [44]:
sum_bld_m3=sum_bld_m3.loc[(sum_bld_m3['precent_of_stat_data']>0.01)|(sum_bld_m3['Taz_num']==2001)]

In [45]:
sum_bld_m3_by_stat=sum_bld_m3.pivot_table(index='STAT',aggfunc=sum)[['bld_m3']].rename(columns={'bld_m3':'bld_m3_stat_new'}).reset_index()

In [46]:
sum_bld_m3=sum_bld_m3.merge(sum_bld_m3_by_stat,on='STAT')

In [47]:
sum_bld_m3['precent_of_stat_data']=sum_bld_m3['bld_m3']/sum_bld_m3['bld_m3_stat_new']

### א"ס ללא פוליגון



In [48]:
stat_in_taz=make_point(stat_in_taz).sjoin(taz[['Taz_num','geometry']])[['STAT','Taz_num']]

In [49]:
stat_in_taz['precent_of_stat_data']=1

### איחוד לטבלה אחת



In [50]:
col=['Taz_num', 'STAT',  'precent_of_stat_data']

In [51]:
precent_of_stat_data=pd.concat([sum_bld_m3[col], stat_in_taz])

In [52]:
precent_of_stat_data.to_excel(r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\output\precent_of_stat_data.xlsx',index=False)

###  יצירת מידע ברמת אזור תנועה

In [53]:
stat=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2020\מרכיבי_תחזית','stat_cbs_jtmt_2020')

In [54]:
col=['pop_0',
 'pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

In [55]:
stat[col]=stat[col].multiply(stat['pop'], axis="index")

In [56]:
stat['pop_hardi']=stat['pop']*(stat['pre_hardi']/100)

In [57]:
taz_stat_conver=precent_of_stat_data

In [58]:
col=['STAT',
 'aprt_20',
 'pop_0',
 'pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',
 'pop',
 'pop_hardi']

בשכבת האזורי תנועה יהיו רק אזורי תנועה שמפצלים את המידע של למס

In [59]:
taz=pd.merge(taz_stat_conver,stat[col],on='STAT',how='left')

In [60]:
col=['aprt_20',
 'pop_0',
 'pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',
 'pop',
 'pop_hardi']

In [61]:
taz[col]=taz[col].multiply(taz['precent_of_stat_data'], axis="index")

In [62]:
taz=pd.pivot_table(taz,index='Taz_num',aggfunc=sum)

In [63]:
taz['pre_hardi']=taz['pop_hardi']/taz['pop']

In [64]:
taz['hh_size']=taz['pop']/taz['aprt_20']

In [65]:
save_taz_path=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2020\מרכיבי_תחזית'

In [66]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [67]:
taz.to_excel(r'{}\taz_with_pop_info_{}.xlsx'.format(save_taz_path,file_date))